In [4]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import math

%matplotlib inline
%run function.ipynb

In [5]:
data = pd.read_csv('model1.csv')
data

,State,Term,NoEmp,NewExist,IsFranchised,UrbanRural,RevLineCr,LowDoc,DisbursementGross,Default,...,DisbursePerEmployee,Bank%,State_division,Is_Cross_State,Appv_Dusburse_delta,RealEstate,Recession,Mon_Is_14710,DisbursePerMonth,rate
0,NC,162.0,2.0,0.0,1.0,1,0.0,0.0,3.071757e+05,0.0,...,84466.666667,25.0,South East,1,52.0,0.0,0.0,0.0,1896.146414,4.47
1,AR,126.0,7.0,1.0,0.0,1,0.0,0.0,1.664374e+05,0.0,...,17162.500000,15.0,South East,1,82.0,0.0,0.0,1.0,1320.931404,4.47
2,CA,83.0,18.0,0.0,0.0,1,1.0,0.0,5.316067e+05,0.0,...,23081.105263,50.0,West,1,21.0,0.0,0.0,0.0,6404.900440,4.47
3,FL,84.0,4.0,1.0,0.0,1,1.0,0.0,6.235643e+04,0.0,...,10288.000000,50.0,South East,0,21.0,0.0,0.0,0.0,742.338435,4.47
4,LA,60.0,3.0,1.0,0.0,1,0.0,0.0,6.061084e+04,0.0,...,12500.000000,15.0,South East,0,113.0,0.0,0.0,0.0,1010.180607,4.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518436,CO,84.0,2.0,1.0,0.0,0,1.0,0.0,3.636650e+04,0.0,...,10000.000000,50.0,South West,1,22.0,0.0,0.0,0.0,432.934546,4.51
518437,AZ,29.0,2.0,0.0,0.0,1,-1.0,0.0,2.060768e+05,1.0,...,56666.666667,25.0,South West,1,22.0,0.0,0.0,0.0,7106.098067,4.51
518438,NV,240.0,20.0,1.0,0.0,1,-1.0,0.0,6.170183e+05,0.0,...,24238.095238,0.0,West,0,464.0,1.0,0.0,0.0,2570.909646,4.51
518439,AZ,84.0,3.0,1.0,0.0,1,1.0,0.0,1.895301e+04,0.0,...,3908.750000,50.0,South West,1,22.0,0.0,0.0,0.0,225.631054,4.51
